# Calculate the log probability of a given sentence based on a corpus of text using bigrams

### Smoothing and logs

There are still a couple of problems to sort out before we use the bigram probability dictionary to calculate the probabilities of new sentences:

1. Some possible combinations may not exist in our probability dictionary but are still possible. We don't want to multiply in a probability of 0 just because our original corpus was deficient. This is solved through "smoothing". There are a number of methods for this, but a simple one is the Laplace smoothing with the "add-one" estimate where VV is the size of the vocabulary for the corpus, i.e. the number of unique tokens:

<img src = "./assets/1.png">

Repeated multiplications of small probabilities can cause underflow problems in computers when
the values become to small. To solve this, we will calculate all probabilities in log space:

<img src = "./assets/2.png">

### Quiz 3 Instructions

In the following quiz, a utility named `utils.bigram_add1_logs` has been added for you with Laplace smoothing in the log space. Write a function that calculates the log probability for a given sentence, using this log probability dictionary. If all goes well, you should observe that more likely sentences yield higher values for the log probabilities.

In [1]:
from collections import Counter
import numpy as np

In [2]:
def sentence_to_bigrams(sentence):
    """
    Add start '<s>' and stop '</s>' tags to the sentence and tokenize it into a list
    of lower-case words (sentence_tokens) and bigrams (sentence_bigrams)
    :param sentence: string
    :return: list, list
        sentence_tokens: ordered list of words found in the sentence
        sentence_bigrams: a list of ordered two-word tuples found in the sentence
    """
    sentence_tokens = ['<s>'] + sentence.lower().split() + ['</s>']
    sentence_bigrams = []
    for i in range(len(sentence_tokens)-1):
        sentence_bigrams.append((sentence_tokens[i], sentence_tokens[i+1]))
    return sentence_tokens, sentence_bigrams

In [3]:
def bigrams_from_transcript(filename):
    """
    read a file of sentences, adding start '<s>' and stop '</s>' tags; Tokenize it into a list of lower case words
    and bigrams
    :param filename: string 
        filename: path to a text file consisting of lines of non-puncuated text; assume one sentence per line
    :return: list, list
        tokens: ordered list of words found in the file
        bigrams: a list of ordered two-word tuples found in the file
    """
    tokens = []
    bigrams = []
    with open(filename, 'r') as f:
        for line in f:
            line_tokens, line_bigrams = sentence_to_bigrams(line)
            tokens = tokens + line_tokens
            bigrams = bigrams + line_bigrams
    return tokens, bigrams

In [4]:
tokens, bigrams = bigrams_from_transcript("./transcript.txt")

In [5]:
tokens

['<s>',
 'go',
 'do',
 'you',
 'hear',
 '</s>',
 '<s>',
 'but',
 'in',
 'less',
 'than',
 'five',
 'minutes',
 'the',
 'staircase',
 'groaned',
 'beneath',
 'an',
 'extraordinary',
 'weight',
 '</s>',
 '<s>',
 'at',
 'this',
 'moment',
 'the',
 'whole',
 'soul',
 'of',
 'the',
 'old',
 'man',
 'seemed',
 'centred',
 'in',
 'his',
 'eyes',
 'which',
 'became',
 'bloodshot',
 'the',
 'veins',
 'of',
 'the',
 'throat',
 'swelled',
 'his',
 'cheeks',
 'and',
 'temples',
 'became',
 'purple',
 'as',
 'though',
 'he',
 'was',
 'struck',
 'with',
 'epilepsy',
 'nothing',
 'was',
 'wanting',
 'to',
 'complete',
 'this',
 'but',
 'the',
 'utterance',
 'of',
 'a',
 'cry',
 '</s>',
 '<s>',
 'and',
 'the',
 'cry',
 'issued',
 'from',
 'his',
 'pores',
 'if',
 'we',
 'may',
 'thus',
 'speak',
 'a',
 'cry',
 'frightful',
 'in',
 'its',
 'silence',
 '</s>',
 '<s>',
 'davrigny',
 'rushed',
 'towards',
 'the',
 'old',
 'man',
 'and',
 'made',
 'him',
 'inhale',
 'a',
 'powerful',
 'restorative',
 '</s>

In [6]:
bigrams

[('<s>', 'go'),
 ('go', 'do'),
 ('do', 'you'),
 ('you', 'hear'),
 ('hear', '</s>'),
 ('<s>', 'but'),
 ('but', 'in'),
 ('in', 'less'),
 ('less', 'than'),
 ('than', 'five'),
 ('five', 'minutes'),
 ('minutes', 'the'),
 ('the', 'staircase'),
 ('staircase', 'groaned'),
 ('groaned', 'beneath'),
 ('beneath', 'an'),
 ('an', 'extraordinary'),
 ('extraordinary', 'weight'),
 ('weight', '</s>'),
 ('<s>', 'at'),
 ('at', 'this'),
 ('this', 'moment'),
 ('moment', 'the'),
 ('the', 'whole'),
 ('whole', 'soul'),
 ('soul', 'of'),
 ('of', 'the'),
 ('the', 'old'),
 ('old', 'man'),
 ('man', 'seemed'),
 ('seemed', 'centred'),
 ('centred', 'in'),
 ('in', 'his'),
 ('his', 'eyes'),
 ('eyes', 'which'),
 ('which', 'became'),
 ('became', 'bloodshot'),
 ('bloodshot', 'the'),
 ('the', 'veins'),
 ('veins', 'of'),
 ('of', 'the'),
 ('the', 'throat'),
 ('throat', 'swelled'),
 ('swelled', 'his'),
 ('his', 'cheeks'),
 ('cheeks', 'and'),
 ('and', 'temples'),
 ('temples', 'became'),
 ('became', 'purple'),
 ('purple', 'as'),

In [7]:
def bigram_add1_logs(transcript_file):
    """
    provide a smoothed log probability dictionary based on a transcript
    :param transcript_file: string
        transcript_file is the path filename containing unpunctuated text sentences
    :return: dict
        bg_add1_log_dict: dictionary of smoothed bigrams log probabilities including
        tags: <s>: start of sentence, </s>: end of sentence, <unk>: unknown placeholder probability
    """

    tokens, bigrams = bigrams_from_transcript(transcript_file)
    token_counts = Counter(tokens)
    bigram_counts = Counter(bigrams)
    vocab_count = len(token_counts)

    bg_addone_dict = {}
    for bg in bigram_counts:
        bg_addone_dict[bg] = np.log((bigram_counts[bg] + 1.) / (token_counts[bg[0]] + vocab_count))
    bg_addone_dict['<unk>'] = np.log(1. / vocab_count)
    return bg_addone_dict

In [8]:
bigram_add1_logs("./transcript.txt")

{('<s>', 'go'): -5.0271645960474665,
 ('go', 'do'): -4.9344739331306915,
 ('do', 'you'): -3.857214768933151,
 ('you', 'hear'): -4.987025428457122,
 ('hear', '</s>'): -4.930870325627393,
 ('<s>', 'but'): -4.334017415487521,
 ('but', 'in'): -4.945207488773801,
 ('in', 'less'): -4.959341999708705,
 ('less', 'than'): -4.930870325627393,
 ('than', 'five'): -4.9344739331306915,
 ('five', 'minutes'): -4.930870325627393,
 ('minutes', 'the'): -4.930870325627393,
 ('the', 'staircase'): -5.062595033026967,
 ('staircase', 'groaned'): -4.930870325627393,
 ('groaned', 'beneath'): -4.930870325627393,
 ('beneath', 'an'): -4.930870325627393,
 ('an', 'extraordinary'): -4.9344739331306915,
 ('extraordinary', 'weight'): -4.930870325627393,
 ('weight', '</s>'): -4.930870325627393,
 ('<s>', 'at'): -5.0271645960474665,
 ('at', 'this'): -4.930870325627393,
 ('this', 'moment'): -4.948759890378168,
 ('moment', 'the'): -4.930870325627393,
 ('the', 'whole'): -5.062595033026967,
 ('whole', 'soul'): -4.930870325627

In [9]:
test_sentences = [
    'the old man spoke to me',
    'me to spoke man old the',
    'old man me old man me',
]

In [10]:
def log_prob_of_sentence(sentence, bigram_log_dict):
    # get the sentence bigrams
    s_tokens, s_bigrams = sentence_to_bigrams(sentence)

    # add the log probabilites of the bigrams in the sentence
    total_log_prob = 0.
    for bg in s_bigrams:
        if bg in bigram_log_dict:
            total_log_prob = total_log_prob + bigram_log_dict[bg]
        else:
            total_log_prob = total_log_prob + bigram_log_dict['<unk>']
    return total_log_prob

In [11]:
def sample_run():
    # sample usage by test code (this definition not actually run for the quiz)
    bigram_log_dict = bigram_add1_logs('transcript.txt')
    for sentence in test_sentences:
        print('*** "{}"'.format(sentence))
        print(log_prob_of_sentence(sentence, bigram_log_dict))

In [13]:
sample_run()

*** "the old man spoke to me"
-34.80495531345013
*** "me to spoke man old the"
-39.34280606002005
*** "old man me old man me"
-36.59899481268447
